In [116]:
include("AuxiliaryFunctions/BornModel.jl")
include("AuxiliaryFunctions/Plotting.jl")

import .BornModel as BM
import .Plotting as P
using Plots, Plots.PlotMeasures
using DifferentialEquations
using BifurcationKit, Parameters

In [118]:
# plot settings

linewidth = 1.5
titlefontsize = 12 
legendfontsize = 10

xmin, xmax = 33.4, 35.5
ymin, ymax = 14.5, 30

(14.5, 30)

## Continuation

In [3]:
parsS2 = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0,
    η = 1.29e2, μ1 = 0.838, μ2 = 83.8, μ3 = 12.1, μ4 = 8.84e-3, r = 0.0714)

x0S2 = [BM.nondimensional_T(6), BM.nondimensional_S(34.5), BM.nondimensional_T(5), BM.nondimensional_S(34.9)]
tspan = (0., 50.)

odeprobS2 = ODEProblem(BM.nondimensional_born!, x0S2, tspan, parsS2)
odesolS2 = solve(odeprobS2)

x0S2 = odesolS2[end];

In [79]:
recordFromSolution(x, p) = (T1 = x[1], S1 = x[2], T3 = x[3], S3 = x[4],
    M = parsS2.r*(1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))) + (1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3]))),
    Δσ13 = (x[2] - x[1]) - (x[4] - x[3]),
    Δσ21 = (p  - parsS2.T2) - (x[2] - x[1]),
    Δσ43 = (parsS2.S4  - parsS2.T4) - (x[4] - x[3]),
    Ud = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])),
    Us = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))
    )

probS2 = BifurcationProblem(BM.nondimensional_born!, x0S2, parsS2, (@lens _.S2); 
    record_from_solution = recordFromSolution
)    

# continuation
opts_newton_c = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_c = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-5, ds=1e-6,
    newton_options = opts_newton_c,
    max_steps = 5e4)

brS2 = continuation(probS2, PALC(), opts_c; 
    detect_bifurcation = 3, 
    bothside = true,
    normC = norminf,
    usedeflation = true)

# diagram
opts_newton_d = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_d = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-6, ds=1e-7,
    newton_options = opts_newton_d,
    max_steps = 1e6) 

dS2 = bifurcationdiagram(probS2, PALC(),
    5,
    bothside = true,
    (args...) -> opts_d,
)

[Bifurcation diagram]
 ┌─ From 0-th bifurcation point.
 ├─ Children number: 0
 └─ Root (recursion level 1)
      ┌─ Curve type: EquilibriumCont
      ├─ Number of points: 1678901
      ├─ Type of vectors: Vector{Float64}
      ├─ Parameter S2 starts at 0.0, ends at 0.01265963541810787
      ├─ Algo: PALC
      └─ Special points:

If `br` is the name of the branch,
ind_ev = index of the bifurcating eigenvalue e.g. `br.eig[idx].eigenvals[ind_ev]`

- #  1, endpoint at S2 ≈ +0.00000000,                                                                      step =   0
- #  2,       bp at S2 ≈ +0.88400540 ∈ (+0.88400540, +0.88400540), |δp|=1e-10, [converged], δ = ( 1,  0), step = 698879, eigenelements in eig[698880], ind_ev =   1
- #  3,       bp at S2 ≈ +0.85751433 ∈ (+0.85751433, +0.85751433), |δp|=5e-11, [converged], δ = ( 1,  0), step = 717737, eigenelements in eig[717738], ind_ev =   2
- #  4,     hopf at S2 ≈ +0.85759615 ∈ (+0.85759613, +0.85759615), |δp|=2e-08, [converged], δ = (-2, -2)

In [119]:
S2_br_lower_stable = dS2.γ.branch[1:698879]
S2_br_lower_unstable = dS2.γ.branch[698880:717802]
S2_br_upper_stable = dS2.γ.branch[717803:858237]
S2_br_upper_unstable = dS2.γ.branch[858238:end];

# plot 0
p0_stable_xvalues1, p0_stable_xvalues2 = BM.dimensional_S(S2_br_lower_stable.param), BM.dimensional_S(S2_br_upper_stable.param)
p0_unstable_xvalues1, p0_unstable_xvalues2 = BM.dimensional_S(S2_br_lower_unstable.param), BM.dimensional_S(S2_br_upper_unstable.param)

p0_stable_yvalues1, p0_stable_yvalues2 = BM.dimensional_M(S2_br_lower_stable.M), BM.dimensional_M(S2_br_upper_stable.M)
p0_unstable_yvalues1, p0_unstable_yvalues2 = BM.dimensional_M(S2_br_lower_unstable.M), BM.dimensional_M(S2_br_upper_unstable.M)

p0 = plot(p0_stable_xvalues1, p0_stable_yvalues1, label = "lower stable branch", dpi = 300,
    legend = :topleft, legendfontsize = legendfontsize, linewidth = linewidth, color = :dodgerblue, ls = :solid, grid = false, titlefontsize = titlefontsize)
plot!(p0_stable_xvalues2, p0_stable_yvalues2, label = "upper stable branch", linewidth = linewidth, color = :dodgerblue, ls = :dashdot)
plot!(p0_unstable_xvalues1, p0_unstable_yvalues1, label = "unstable branch", linewidth = linewidth, color = :grey65)
plot!(p0_unstable_xvalues2, p0_unstable_yvalues2, label = "", linewidth = linewidth, color = :grey65)
P.plot_special_points(dS2.γ, :S2, :M; plot_legend = true)

xlims!(xmin, xmax)
ylims!(ymin, ymax)

title!("\n a) M")
xlabel!("S\$_2\$ [psu]")
ylabel!("M [Sv]")


# plot 1
p1_xvalues1, p1_xvalues2 = BM.dimensional_S(S2_br_lower_stable.param), BM.dimensional_S(S2_br_upper_stable.param)
p1_yvalues1, p1_yvalues2 = BM.dimensional_σ(S2_br_lower_stable.Δσ13), BM.dimensional_σ(S2_br_upper_stable.Δσ13)

p1 = plot(p1_xvalues1, p1_yvalues1, label = "", dpi = 300, yticks = -1.5:0.5:0.5,
    linewidth = linewidth, color = :dodgerblue,  grid = false, titlefontsize = titlefontsize) #"non-convective"
plot!(p1_xvalues2, p1_yvalues2, label = "", linewidth = linewidth, color = :dodgerblue, ls = :dashdot) #"convective"
P.plot_special_points(dS2.γ, :S2, :Δσ13; plot_legend = false)
ylabel!("\$ Δσ\$ [kg m\$^{-3}\$]") 
title!("\n b) \$σ_{1} - σ_{3}\$ ")
xlims!(xmin, xmax)
ylims!(-1.6, 0.5)

# plot 2
p2_xvalues1, p2_xvalues2 = BM.dimensional_S(S2_br_lower_stable.param), BM.dimensional_S(S2_br_upper_stable.param)
p2_yvalues1, p2_yvalues2 = BM.dimensional_σ(S2_br_lower_stable.Δσ21), BM.dimensional_σ(S2_br_upper_stable.Δσ21)

p2 = plot(p2_xvalues1, p2_yvalues1, label = "", dpi = 300, yticks = -1.5:0.5:0.5,
    linewidth = linewidth, color = :dodgerblue,  grid = false, titlefontsize = titlefontsize)
plot!(p2_xvalues2, p2_yvalues2, label = "", linewidth = linewidth, color = :dodgerblue, ls = :dashdot)
P.plot_special_points(dS2.γ, :S2, :Δσ21, plot_legend = false)
ylabel!("\$ Δσ\$ [kg m\$^{-3}\$]") 
title!("c) \$σ_{2} - σ_{1}\$")
xlims!(xmin, xmax)
ylims!(-1.6, 0.0)
# yticks!([-0.02:0.02:0.04;])

# plot 3
p3_xvalues1, p3_xvalues2 = BM.dimensional_S(S2_br_lower_stable.param), BM.dimensional_S(S2_br_upper_stable.param)
p3_yvalues1, p3_yvalues2 = BM.dimensional_σ(S2_br_lower_stable.Δσ43), BM.dimensional_σ(S2_br_upper_stable.Δσ43)

p3 = plot(p3_xvalues1, p3_yvalues1, label = "", dpi = 300, yticks = -0.12:0.03:0.03,
    linewidth = linewidth, color = :dodgerblue,  grid = false, titlefontsize = titlefontsize)
plot!(p3_xvalues2, p3_yvalues2, label = "", linewidth = linewidth, color = :dodgerblue, ls = :dashdot)
P.plot_special_points(dS2.γ, :S2, :Δσ43, plot_legend = false)
ylabel!("\$ Δσ\$ [kg m\$^{-3}\$]") 
title!("d) \$σ_{4} - σ_{3}\$")
xlims!(xmin, xmax)
ylims!(-0.10, 0.03)
# yticks!([-0.12:0.03:0;])
xlabel!("S\$_2\$ [psu]")


###############
# combining in one plot

layout = @layout [a{0.5w} grid(3,1){0.5w}]

plot(p0, p1, p2, p3, layout=layout, 
    size = (width = 1000, height = 500), # (800, 400)
    bottom_margin = 25px, left_margin = 20px, top_margin = 10px, right_margin = 20px)

savefig("Figures/branches.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\branches.pdf"